In [37]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.python.keras.backend as K

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

from helpers.utils import get_public_key, get_private_key, NumpyEncoder, NumpyDecoder, get_dataset, fetch_index

In [38]:
def encrypt_message(message, recipient_public_key):
    ephemeral_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    ephemeral_public_key = ephemeral_private_key.public_key()

    shared_secret = ephemeral_private_key.exchange(ec.ECDH(), recipient_public_key)

    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=32 + 12,
        salt=None,
        info=b'',
    ).derive(shared_secret)

    encryption_key = derived_key_material[:32]
    nonce = derived_key_material[32:]

    cipher = AESGCM(encryption_key)
    ciphertext = cipher.encrypt(nonce, message.encode(), None)

    return ephemeral_public_key, ciphertext


def decrypt_message(ciphertext, ephemeral_public_key, recipient_private_key):
    shared_secret = recipient_private_key.exchange(ec.ECDH(), ephemeral_public_key)

    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=32 + 12,
        salt=None,
        info=b'',
    ).derive(shared_secret)

    encryption_key = derived_key_material[:32]
    nonce = derived_key_material[32:]

    cipher = AESGCM(encryption_key)
    decrypted_message = cipher.decrypt(nonce, ciphertext, None)

    return decrypted_message.decode()

# 
# recipient_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
# recipient_public_key = recipient_private_key.public_key()
# 
# message_to_encrypt = "Hello, this is plaintext"
# 
# ephemeral_public_key, ciphertext = encrypt_message(message_to_encrypt, recipient_public_key)
# 
# print("Ciphertext:", ciphertext)
# 
# decrypted_message = decrypt_message(ciphertext, ephemeral_public_key, recipient_private_key)
# print("Decrypted Message:", decrypted_message)

In [109]:
indexes = fetch_index("mnist")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, y_train, x_test, y_test = get_dataset(indexes[0], "mnist", x_train, y_train, x_test, y_test)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu', padding='same'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(x_train, y_train, epochs=1, batch_size=100, verbose=True)  #  history.history['loss']
loss_perturbed = model.evaluate(x_test, y_test, verbose=0)[0]

12/12 [==============================] - 2s 71ms/step - loss: 1.1147 - accuracy: 0.6400


In [67]:
json_str = json.dumps(model.get_weights(), cls=NumpyEncoder)

private_key = get_private_key(1, 'elliptical')
public_key = get_public_key(1, 'elliptical')

ephemeral_public_key, ciphertext = encrypt_message(json_str, public_key)
decrypted_message = decrypt_message(ciphertext, ephemeral_public_key, private_key)
decrypted_weights = json.loads(decrypted_message, cls=NumpyDecoder)

In [5]:
def get_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model


@tf.function
def get_logits(model, x):
    return model(x)


@tf.function
def loss_fn(logits, y):
    return K.categorical_crossentropy(y, logits)


def random_weight_selection(weights, fraction=0.25):
    percentage = max(0, min(100, fraction))
    flattened_weights = weights.flatten()
    num_elements = int(np.ceil(percentage * flattened_weights.size))
    indexes = np.random.choice(flattened_weights.size, size=num_elements, replace=False)
    original_indices = np.unravel_index(indexes, weights.shape)
    indices = [arr.tolist() for arr in original_indices]
    return indices


def magnitude_weight_selection(array, percentage=0.25):
    percentage = max(0, min(100, percentage))
    num_elements = int(np.ceil(percentage / 100 * array.size))
    indices_of_largest = np.argpartition(array.flatten(), -num_elements)[-num_elements:]
    original_indices = np.unravel_index(indices_of_largest, array.shape)
    indices = [arr.tolist() for arr in original_indices]
    return indices


def obd_weight_selection(model, x, y, weights, fraction):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, weights)
        sensitivities = [grad * weight for grad, weight in zip(gradients, weights)]
        sensitivities = np.array(sensitivities)
        return magnitude_weight_selection(sensitivities, fraction)
    
def regularization_weight_selection(model, x, y, reg_type, weights, fraction):
    regularization_lambda = 0.01
    l1_regularization = regularization_lambda * tf.reduce_sum(tf.abs(weights))
    l2_regularization = regularization_lambda * tf.reduce_sum(tf.square(weights))

    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, weights)

    total_gradient = gradients + (l1_regularization if reg_type == "l1" else l2_regularization)
    l1_weight = np.array(weights - total_gradient)
    return magnitude_weight_selection(l1_weight, fraction)